## 1. 什么是 Peptide Fragmentation？（肽段断裂）

在蛋白质组学里，我们想知道肽段的氨基酸序列。但直接测序很难，于是我们采用一个“曲线救国”的办法：

> 把肽段打碎成更小的碎片，再通过每个碎片的质量（m/z）**反推出原始序列**。

这个打碎的过程就叫 **fragmentation（断裂）**，发生在 **MS/MS（又叫 MS2）** 阶段，是质谱仪的二次分析。

- CID 模式下的过程像这样：

1. **先挑出一个肽段（precursor ion）**  
   → 在 MS1 阶段，质谱仪选中一个带电的肽段，比如 "PEPTIDE"

2. **把它丢进“撞墙房”**  
   → 用氩气/氮气猛烈撞击肽段，断在**肽键（C–N）**上

3. **测碎片的质量**  
   → 每个断裂点会形成一个小的“带电碎片离子”，我们就能根据这些碎片的 m/z 推出“这段肽原来长这样”

💥 这就像炸开一根项链，再看每段珠子链的重量，猜出整串项链的顺序！

---

## 2. b / y 离子是怎么来的？

CID 最常见的断裂方式是：
- 在氨基酸之间的 **C–N 键**断开
- 会生成两种碎片：
  - 从 N 端那边过来的叫 **b 离子**
  - 从 C 端那边过来的叫 **y 离子**

![peptide fragmentation](pics/frag.png)

比如这个肽段：
```
PEPTIDE
```

它断裂后会变成：

```
N端（顺序）：
b1   b2   b3   b4   b5   b6
P -  PE - PEP - PEPT - PEPTI - PEPTID

C端（倒序）：
y1   y2   y3   y4   y5   y6
E -  DE - IDE - TIDE - PTIDE - EPTIDE
```

| 离子类型 | 来源方向 | 包含什么质量 | 在谱图里常见吗？ |
|----------|----------|---------------|-------------------|
| b 离子   | 从 N 端  | N 端部分 + H⁺ | ✔ 超常见          |
| y 离子   | 从 C 端  | C 端部分 + H₂O + H⁺ | ✔ 超常见     |


### 为什么 b 离子是 N 端 + H⁺、y 离子是 C 端 + H₂O + H⁺？

#### 一、首先，碎片必须是 **带电的（ion）** 才能被 MS 探测！

质谱仪只能“看到”带电粒子。碎片如果没电荷，它就像隐形人一样飘过去你也看不到。

所以在断裂之后，**必须有一个碎片能抓住质子（H⁺）【是一种带有1个单位电荷正电的稳定强子，通常标记为 p,p+或H+】或者保留原有电荷**，这叫 **protonation**。


#### 那为什么是 b 离子 + H⁺，y 离子 + H₂O + H⁺？

来个最通俗的大白话总结：

| 离子类型 | 化学起源 | 多出来的质量来源 | 为什么它合理？ |
|----------|------------|--------------------|----------------|
| **b 离子** | N 端残留部分 | 加 1 个质子 (H⁺) → **+1.0073 Da** | 因为它是一个有游离氨基（–NH₂）的一端，非常容易质子化，带正电稳定 |
| **y 离子** | C 端残留部分 | 加 1 个水分子 (H₂O) + 1 个质子 → **+19.0184 Da** | y 离子的 C 端本来就是个羧基（–COOH），fragment 后重新水合，加水形成稳定末端结构 |

##### b 离子结构（N-terminal）
- 断裂后，b 离子是一段 **带氨基的肽链**
- 这个氨基可以接受一个 H⁺ 

👉 所以加上 H⁺ 是符合氨基的化学性质，也方便电荷稳定存在于 N 端


##### y 离子结构（C-terminal）
- y 离子那边含有羧基（–COO⁻）的末端
- 断裂后，这个末端会与 H⁺ 和 OH⁻ 重组，形成一个**完整的羧基末端（–COOH）**，等于加了 **1 个 H₂O**
- 然后再接受一个 H⁺ 带正电

👉 所以 y 离子的“C 端 + H₂O + H⁺”也非常合理，从断裂化学 +电荷位置 都讲得通！

#### 质量总结：

| 离子类型 | 额外加的质量（Da） | 来源         |
|----------|---------------------|--------------|
| b 离子   | +1.0073             | 质子 H⁺      |
| y 离子   | +18.0106 + 1.0073 = **+19.018** | 水 (H₂O) + H⁺ |

---


## 用 Python 生成碎片质量！

In [5]:
pip install pyteomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 6.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyteomics import mass

def get_b_y_ions(sequence):
    b_ions = []
    y_ions = []
    for i in range(1, len(sequence)):
        b_seq = sequence[:i]
        y_seq = sequence[i:]
        b_mass = mass.calculate_mass(sequence=b_seq, ion_type='b')
        y_mass = mass.calculate_mass(sequence=y_seq, ion_type='y')
        b_ions.append((f"b{i}", b_mass))
        y_ions.append((f"y{len(sequence)-i}", y_mass))
    return b_ions, y_ions

# Demo
b, y = get_b_y_ions('PEPTIDE')
b + y


[('b1', 97.05276384884999),
 ('b2', 226.09535693682),
 ('b3', 323.14812078566996),
 ('b4', 424.19579925408),
 ('b5', 537.27986323121),
 ('b6', 652.30680625504),
 ('y6', 702.30720017786),
 ('y5', 573.2646070898901),
 ('y4', 476.21184324104),
 ('y3', 375.16416477263),
 ('y2', 262.0801007955),
 ('y1', 147.05315777167)]

# Low Energy CID
In low energy CID (i.e. collision induced dissociation in a triple quadrupole or an ion trap) a peptide carrying a positive charge fragments mainly along its backbone, generating predominantly b and y ions. In addition, for fragments containing RKNQ, peaks are seen for ions that have lost ammonia (-17 Da) denoted a*, b* and y*. For fragments containing STED, loss of water (-18 Da) is denoted a°, b° and y°. Satellite ions from side chain cleavage are not observed.

# High Energy CID
(As in 4 sector instruments, not HCD in a trap)

All of the ion series described above are observed in high energy collision spectra. Relative abundances are composition dependent. Unlike low energy CID, ions do not readily lose ammonia or water.

The characteristic effects of individual amino acid residues on peptide fragmentation behaviour have been reviewed in detail by Papayannopoulos [Papayannopoulos, 1995].

# HCD
Higher-energy collisional dissociation (HCD) is a CID technique specific to the orbitrap mass spectrometer in which fragmentation takes place external to the trap.[8] HCD was formerly known as higher-energy C-trap dissociation. In HCD, the ions pass through the C-trap and into the HCD cell, an added multipole collision cell, where dissociation takes place. The ions are then returned to the C-trap before injection into the orbitrap for mass analysis. HCD does not suffer from the low mass cutoff of resonant-excitation (CID) and therefore is useful for isobaric tag–based quantification as reporter ions can be observed. Despite the name, the collision energy of HCD is typically in the regime of low energy collision induced dissociation (less than 100 eV).[8][9]

# References
- https://www.matrixscience.com/help/fragmentation_help.html
- https://en.wikipedia.org/wiki/Collision-induced_dissociation


### **相同点 Similarities**
| 项目 | 描述 |
|------|------|
| 本质 | 都是 **collision-induced dissociation**（碰撞诱导解离），靠和惰性气体碰撞打碎带电肽段 |
| 目的 | 生成结构信息——尤其是肽段骨架碎片（b, y 离子）用于肽段序列分析 |
| 应用 | 都能用于蛋白质组学的质谱分析（尤其是 MS/MS） |


### ❗️**不同点 Differences**

| 特征 | **Low Energy CID** | **High Energy CID (传统)** | **HCD (Orbitrap专属)** |
|------|---------------------|-----------------------------|--------------------------|
| 设备平台 | ion trap, triple quadrupole | 四重极–磁扇形（4-sector）仪器 | Orbitrap 系统（Thermo 系） |
| 碰撞能量 | 低（通常 <100 eV）| 高（>100 eV）| 中等，**虽然叫 High，但其实还是低能级 CID (<100 eV)** |
| 解离位置 | **主碎 b, y 离子**（肽键断裂）| 更多类型的离子（a, b, c, x, y, z）| **主碎 b, y 离子**，但能看见更多小离子 |
| 氨/水丢失 | **常见：R, K, N, Q → NH₃ 丢失；S, T, E, D → H₂O 丢失** | **不常见** | 相对较少，但仍可能存在 |
| 是否有低质量离子截断（low-mass cutoff） | 有（尤其在 ion trap）| 无 | **无（可见低质量 reporter ion）**，对 **TMT/iTRAQ** 特别重要 |
| 离子收集方式 | trap 内碎 → trap 内收集 | 外部碎 → 外部收集 | **Trap 外碎 → 回 trap 分析** |


### 通俗理解方式

想象你在打爆一个乐高机器人（蛋白质），然后收集掉落的零件（碎片）来推测它原本长啥样：

- **CID（Low Energy）**：你在屋子（trap）里打，用橡皮锤敲。只敲断主要结构，掉的多是“大件”（b/y 离子），有些零件（NH₃, H₂O）还会自己飞掉。
- **High Energy CID**：你在外面用大铁锤猛砸（像古早的4-sector），全身都能碎成各种乱七八糟的东西，包括一堆小零件。但不容易再有自发掉 NH₃/H₂O 的情况。
- **HCD**：名字虽然叫 high energy，但其实就是在外面温柔地敲（外部cell），再把碎片送回来分析。和 CID 的橡皮锤差不多，但因为不是在 trap 里敲，所以能保住更多小零件——比如 **TMT标签**那种重量很轻的 reporter ion，trap 里看不到的，它能看到！


### 实战角度怎么选？

- 如果你是做 **定量质谱（TMT/iTRAQ）**，**必须用 HCD**，因为需要低质量区的 reporter ions。
- 如果是 **常规肽段鉴定**，CID 或 HCD 都行，不过 Orbitrap 上基本默认就是 HCD。
- 讲究碎片多样性？那就上 **高能 CID（老式）**，但现代仪器已经比较少用了。


## Trap

### 通俗定义：
**Trap（离子阱）** 是一种可以**暂时捕获并控制带电粒子**（离子）的装置，就像你在打游戏时把敌人困在一个小房间里，先不打，等技能CD好了再开打。

### 具体作用：

| 功能 | 说明 |
|------|------|
| **储存离子** | 让离子“在里面等着”，等你准备好了再释放 |
| **控制碰撞** | 你可以把碰撞气体（比如氮气）放进去，在里面敲打离子产生碎片（这就是 low energy CID） |
| **筛选/排序** | 可以按质荷比（m/z）挑选特定的离子分析 |
| **时间控制** | 可以决定离子停留多久，慢慢提高灵敏度和分辨率 |


### 常见的 trap 类型：

| Trap 类型 | 简要说明 |
|-----------|----------|
| **Linear ion trap (LIT)** | 最常见，像个电场滑梯，离子在里面被电场束缚 |
| **3D ion trap** | 早期的版本，用环形电极围住离子（球形困住） |
| **Orbitrap** | 不算传统“trap”，但离子也会被束缚在中央电场中转圈 |
| **FT-ICR trap** | 超高端，用磁场和电场把离子困在一个超稳定轨道上 |


### 为啥 "trap" 有限制？

比如：
- **Ion trap CID** 有个 **low-mass cutoff** —— 意思是太轻的小离子飞不出来（就像风太大，小纸片飞不出窗户）
- 所以有些重要的小离子（比如 TMT 的 reporter ion）在 trap 里 **看不到**
- 这也是 HCD 后来设计成在 trap **外面碎离子** 的原因之一！



1. 蛋白质组学经典ms2中ms1 和 ms2 作用？
2. 质谱仪如何通过 ms2 来识别肽段序列
3. 在 tmt based proteomics 如何使用 tmt 对不同样本的蛋白质定量
4. tmt 工作原理
5. 质谱仪如何产生 tmt 报告离子 reporter ions 

## 1. **MS1 和 MS2 的作用**

在经典的蛋白质组学数据依赖型采集（Data-Dependent Acquisition, DDA）流程中，质谱仪通过两级串联质谱（MS/MS）实现肽段识别：

- **MS1（一级质谱）** 用于对肽段离子（precursor ions）进行扫描，记录其质荷比（m/z）和丰度。MS1 提供整体离子图谱，并用于选择待碎裂的目标离子。
- **MS2（二级质谱）** 针对从 MS1 中选中的某个肽段离子，进行碰撞诱导解离（CID 或 HCD），生成结构相关的碎片离子（fragment ions），从而获得其序列信息。

MS1 提供前体离子的信息；MS2 提供结构信息，是实现蛋白质识别的关键步骤。


## 2. **MS2 如何识别肽段序列**

MS2 利用碰撞诱导解离（CID 或 HCD）打断肽链主链（通常是肽键）生成一系列有规律的碎片离子，主要包括：

- **b 离子**：从肽段的 N 端起始；
- **y 离子**：从 C 端起始。

通过测量 b/y 离子的质荷比（m/z），计算相邻离子的质量差，可以反推出各氨基酸残基的质量，从而还原肽段序列。现代数据库搜索软件（如 SEQUEST、Mascot、MaxQuant）会将实验获得的 MS2 图谱与理论图谱匹配，进行序列鉴定。


## 3. **TMT 在蛋白质定量中的原理和作用**

TMT（Tandem Mass Tag）是一类同分异构的稳定同位素标签，用于实现多样本相对定量。TMT 标签可通过化学修饰方式共价连接于肽段的 N 端或赖氨酸（Lys）残基。

- TMT 标记后的不同样本肽段在 MS1 扫描中具有**相同的 m/z**（因为标签经过质量均衡设计），从而在 MS1 水平无法区分。
- 在 MS2 中，TMT 标签碎裂并释放出**不同质量的 reporter ions（报告离子）**，这些离子质量是固定的（例如 TMT10plex 对应的 126–131 m/z）。
- 各 reporter ion 的相对强度反映了同一肽段在不同样本中的丰度。


## 4. **TMT 的工作机制**

TMT 标签由三部分组成：

- **Reporter group（报告离子部分）**：在 MS2 中释放出用于定量的低质量离子（如 126–131 Da）；
- **Balancer group（质量均衡部分）**：与 reporter 共同调控整体标签质量，使不同标签的总质量一致；
- **Reactive group（活性部分）**：与肽段的 N 端或赖氨酸残基发生共价反应，实现标记。

TMT 标记后的肽段在一级质谱中表现为相同的 m/z；在二级质谱中通过标签的特异性碎裂释放 reporter ions，从而实现样本间的区分与定量。

![TMT labeling](pics/TMT.png)

### TMT 到底是怎么“贴”在肽段上的？

> TMT 的 **reactive group** 是一种 **NHS 酯（N-hydroxysuccinimide ester）**，它专门和肽段中的 **一级胺（–NH₂）** 发生共价键合，最终形成一个 **酰胺键**。


### 那一级胺在哪里？

- 肽段的 **N 端氨基**：开头的 –NH₂
- **赖氨酸（Lys）残基的侧链胺**：–CH₂–CH₂–CH₂–CH₂–NH₂

这两个位置都有自由的胺基，是 NHS 酯攻击的目标。

### 🧪 反应机制

#### Step 1：准备工作  
蛋白质被酶（如 trypsin）消化后变成肽段，肽段的 N 端和每个 Lys 侧链都有 –NH₂。

#### Step 2：TMT 加入  
加入 TMT 标签，它的 **reactive group 是一个 NHS 酯**，有很强的亲核反应性，专门找胺基打交道。

#### Step 3：反应发生  
NHS 酯和胺基发生亲核取代反应，**形成一个稳定的酰胺键**，把 TMT 标签牢牢贴在肽段的 N 端或 Lys 上。

- 化学式大致如下：
$$
\text{TMT-NHS} + \text{R-NH}_2 \rightarrow \text{TMT-NH-C(=O)-R} + \text{NHS}
$$

### 🔁 结果：

一个肽段最终可能有多个 TMT 标签，具体取决于它有几个 Lys。**每一个标记位点都能释放出 reporter ion。**

**例子**：  
肽段序列：`PEPTIDEK`  
Trypsin 消化后，N 端是 `P`，末尾 `K` 有侧链胺  
→ 会被 TMT 标记两个位置：**N 端 + K**


## 5. **质谱仪如何产生 reporter ions**

在 MS2 阶段（尤其使用 HCD 模式）：

1. 被选中的肽段离子（带有 TMT 标签）进入碰撞池，与惰性气体（如氮气）发生碰撞；
2. 除了肽链主链发生断裂生成 b/y 离子外，TMT 标签也会定向断裂；
3. 此断裂释放出小分子 reporter ion，其质量取决于所用的 TMT 标签版本（如 126.1277 Da, 127.1248 Da 等）；
4. 通过检测不同 reporter ion 的强度，可以推断不同样本中该肽段的相对表达量。


## 总结

- **MS1** 提供肽段离子的质量信息，用于前体选择；
- **MS2** 通过碎片分析实现肽段序列鉴定；
- **TMT** 实现多样本肽段定量，其标签在 MS1 中不可区分，在 MS2 中通过 reporter ion 完成样本区分；
- **质谱仪** 通过高能碰撞诱导标签断裂，生成 reporter ion，从而实现定量分析。
